###### Imports

In [1]:
import time
from datetime import datetime, timedelta
import pandas as pd
import glob
import sys
import numpy as np
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
from functions import epoch

[data source](<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" href="http://purl.org/dc/dcmitype/Dataset" property="dct:title" rel="dct:type">Tennis databases, files, and algorithms</span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://www.tennisabstract.com/" property="cc:attributionName" rel="cc:attributionURL">Jeff Sackmann / Tennis Abstract</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/JeffSackmann" rel="dct:source">https://github.com/JeffSackmann</a>)

Since the data license is non-commercial, this is a proof-of-concept project and cannot be immediately applied to commercial use.

In [12]:
# read in data
atp_matches = glob.glob('../tennis_atp/atp_matches*.csv')

# separate out by match category
matches_tour = [file for file in atp_matches if 
                       len(file) == len('../tennis_atp/atp_matches_2019.csv')]
matches_qual_chall = glob.glob('../tennis_atp/atp_matches_qual_chall_*.csv')
matches_futures = glob.glob('../tennis_atp/atp_matches_futures_*.csv')
rankings = glob.glob('../tennis_atp/atp_rankings_*.csv')
players = glob.glob('../tennis_atp/atp_players.csv')
matches_amateur = glob.glob('../tennis_atp/atp_matches_amateur.csv')

# singles matches file list
singles_matches = matches_qual_chall + matches_futures + matches_tour + matches_amateur
# Combined DataFrames
singles_matches_df = pd.concat((pd.read_csv(f) for f in singles_matches), ignore_index=True)
# select columns to be used in determining glicko2 ratings
matches_glicko2 = singles_matches_df[['tourney_date','winner_id','loser_id','tourney_id']]
# Separate Dataframes
players_df = pd.read_csv(players[0]) 
rankings_df = pd.concat((pd.read_csv(ranks) for ranks in rankings))
matches_qual_chall_df = pd.concat((pd.read_csv(matches) for matches in matches_qual_chall))
matches_futures_df = pd.concat((pd.read_csv(matches) for matches in matches_futures))
matches_tour_df = pd.concat((pd.read_csv(matches) for matches in matches_tour))
matches_amateur_df = pd.read_csv(matches_amateur[0])

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_1256/1076576185.py:16: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  singles_matches_df = pd.concat((pd.read_csv(f) for f in singles_matches), ignore_index=True)
/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_1256/1076576185.py:25: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  matches_amateur_df = pd.read_csv(matches_amateur[0])


Date formatting [source]('https://stackoverflow.com/questions/9750330/how-to-convert-integer-into-date-object-python')

In [13]:
# matches_glicko2['tourney_date'] = matches_glicko2['tourney_date'].apply(
#     lambda x: pd.to_datetime(str(x), format='%Y%m%d')) # convert the tourney_date to datetime
matches_glicko2.sort_values(by = 'tourney_date', inplace = True) # sort the rows my tourney_date
matches_glicko2.index = [x for x in range(matches_glicko2.shape[0])] # adjust the index

/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_1256/2950824109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_glicko2.sort_values(by = 'tourney_date', inplace = True) # sort the rows my tourney_date


In [14]:
matches_glicko2.to_csv('../Data/matches_glicko2.csv',index=False)

All matches have a winner and loser id, which indicates that all the datapoints can be used for determining the elo ranking.  There is a good amount of standing rank data, suggesting that this data can be used for testing the relative strength of standard ranking vs elo rating in predicting outcomes.

Idea: map real rankings to expected elo

In [8]:
def pct_nan(df):
    '''take in a dataframe, compute the percentage of values that are not 
    missing for each column, and return as a series'''
    pct_nan = pd.Series(df.shape[0] - df.isna().sum().values)/(df.shape[0])
    pct_nan.index = df.isna().sum().index
    return pct_nan.sort_values(ascending = False)

In [9]:
pct_nan(singles_matches_df);

In [158]:
# concept for generating player ids as strings 
# [f'p_{str(player)}' for player in list(m_1900['loser_id'].values)]

In [157]:
# concept for class instantiation
# {player: Player() for player in [f'p_{str(player)}' for player in list(m_1900['loser_id'].values)]}


/var/folders/7q/63cgxnn50sl5zwxkchv_j2cr0000gn/T/ipykernel_8779/4114524296.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches_glicko2.sort_values(by = 'tourney_date', inplace = True)


In [117]:
epoch2(matches_glicko2.iloc[0:1000,:],[],{})

([114688,
  114689,
  114690,
  114691,
  114692,
  114693,
  116737,
  114695,
  116743,
  114697,
  116745,
  114699,
  116747,
  116740,
  114702,
  116751,
  114704,
  116753,
  114701,
  114707,
  114708,
  114709,
  114710,
  114711,
  114712,
  114713,
  114714,
  114715,
  116764,
  114717,
  116766,
  114718,
  114716,
  114721,
  114720,
  114723,
  114724,
  114725,
  114726,
  114727,
  114728,
  114729,
  114730,
  114731,
  114732,
  114733,
  114734,
  113984,
  115764,
  114768,
  115793,
  114003,
  114628,
  116881,
  116882,
  116883,
  116884,
  116885,
  116886,
  116887,
  116888,
  116889,
  116890,
  116891,
  116892,
  116893,
  116894,
  116895,
  116896,
  116897,
  116898,
  116899,
  116900,
  116901,
  116902,
  116903,
  116904,
  116905,
  116906,
  116907,
  116908,
  116909,
  116910,
  116911,
  116912,
  116913,
  116914,
  116915,
  116916,
  116917,
  116918,
  116919,
  116920,
  116921,
  116922,
  116923,
  116924,
  116925,
  116926,
  116927,


In [65]:
[x for x in range(0,6,2)]

[0, 2, 4]

In [92]:
matches_glicko2[matches_glicko2['tourney_date']<matches_glicko2['tourney_date'][851993]]

,tourney_date,winner_id,loser_id,tourney_id


In [108]:
matches_glicko2['tourney_date'][2]

Timestamp('1877-07-09 00:00:00')

In [113]:
match_history = matches_glicko2[matches_glicko2['tourney_date'] <= matches_glicko2['tourney_date'][100]]
max_date = max(match_history['tourney_date']) # maximum date in the records
min_date = min(match_history['tourney_date']) # minimum date in the records
date_range = range(0,(max_date-min_date).days + 1,365)
[d for d in date_range]

[0, 365, 730]

In [102]:
def epochs(match_history,interval_length = 365):
    '''Calculate the ending rating for each player with the lengh of each epoch being
    a funtion of the interval_length'''
    
    max_date = max(match_history['tourney_date']) # maximum date in the records
    min_date = min(match_history['tourney_date']) # minimum date in the records
    date_range = range(0,(max_date-min_date).days + 1,interval_length) # days from date of first
    # match in increments of the interval (default 365).  This is the length of each epoch. 
    players_list = []
    players_dict = {}
    
    epoch_cutoffs = [min_date + timedelta(days = x) for x in date_range]
    print(epoch_cutoffs)
    epoch_ranges = zip(epoch_cutoffs[0:-2],epoch_cutoffs[1:-1])
    
    
    # iteratively re-update for each epoch
    for period in epoch_ranges:
        rating_period = match_history[(match_history['tourney_date']>=period[0])&(match_history['tourney_date']<period[1])]
        if not rating_period.empty:
            (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
        else:
            for player in players_list:
                players_dict[player].did_not_compete()
    # get the final, extended rating period updates.
    rating_period = match_history[match_history['tourney_date']>=epoch_cutoffs[-1]]
    (players_list,players_dict) = epoch(rating_period,players_list,players_dict)
    return (players_list,players_dict)
    

In [ ]:
start = time.time()
epochs(cutDf(matches_glicko2,756977,805977,'tourney_date'),365)
compute_time = time.time()-start
print(compute_time)

In [19]:
matches_glicko2.shape; # the shape is (876978, 4)

In [35]:
(876977-800977)/876977

0.08666133775458193

In [48]:
(876977-756977)/876977

0.13683369119144514

In [61]:
matches_glicko2.loc[matches_glicko2.index[756977]:matches_glicko2.index[787977],:]

,tourney_date,winner_id,loser_id,tourney_id
40162,2017-11-20,105302,111187,2017-7808
40161,2017-11-20,111200,122617,2017-7808
40160,2017-11-20,104873,106022,2017-7808
40159,2017-11-20,105938,144935,2017-7808
40158,2017-11-20,105738,126523,2017-7808
...,...,...,...,...
218339,2018-12-31,200587,106250,2019-M-ITF-USA-01A-2019
218340,2018-12-31,105654,207968,2019-M-ITF-USA-01A-2019
218341,2018-12-31,200516,122248,2019-M-ITF-USA-01A-2019
218342,2018-12-31,111583,126087,2019-M-ITF-USA-01A-2019


In [126]:
from functions import epoch

ImportError: cannot import name 'epoch' from 'functions' (/Users/williammohr/dsir/capstone/Notebooks/functions.py)

from match index 806977 to match index 876977, or 8% of the data, it takes 4 seconds.
13% of the data (756977 to 876977) takes 11 seconds.

In [ ]:
start = time.time()
epochs(matches_glicko2[matches_glicko2.tourney_date<
                matches_glicko2.tourney_date
                [matches_glicko2.tourney_date.index[100]]],365)
compute_time = time.time()-start
print(compute_time)

In [ ]:
# 23 seconds for first 10_000 matches

The above will updating the ratings and rating deviations for all players 

efficient solution to creating one df from multiple files is from [Sid](https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe) 